# Tech Job Market and Salaries Analysis 

For our final project, we have selected the Stack Overflow Developer Survey dataset, 
which contains detailed responses from developers regarding their job roles, skills, 
technologies used, and salary information. This dataset is particularly relevant to the 
tech industry, which is a major focus of our group, and will provide insights into the tech 
job market by collecting responses from developers worldwide. It covers various topics 
such as job roles, salary, coding activities, education, technology usage, and job 
satisfaction.<br>

Team Eyy<br>
Members:  
- Julianne Kristine D. Aban 
- Derich Andre G. Arcilla 
- Jennifer Bendoy 
- Richelle Ann C. Candidato 
- Marc Francis B. Gomolon 
- Phoebe Kae A. Plasus

##### Data Preparation

LOADING DATA SET & LIBRARIES

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
# df = pd.read_csv('survey_results_filtered.csv')
df = pd.read_csv('survey_results_public.csv')
df.head()

In [ ]:
# Expand display settings to show all columns
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)     # Adjust rows if needed


In [ ]:
# Display column information: name, number of missing values, and dtype
column_info = pd.DataFrame({
    'Column Name': df.columns,
    'Missing Values': df.isnull().sum(),
    'Data Type': df.dtypes
}).reset_index(drop=True)

# Print the column information
print(column_info)

In [ ]:
# Calculate the percentage of missing values
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Filter columns with more than 50% missing values
high_missing_cols = missing_percentage[missing_percentage > 50]
print("Columns with more than 50% missing values:")
print(high_missing_cols)


In [ ]:
# Drop columns with more than 50% missing values
df_cleaned = df.drop(columns=high_missing_cols.index)
print(f"Dataset shape after dropping columns: {df_cleaned.shape}")

# Show the names of the remaining columns
remaining_columns = df_cleaned.columns
print(f"Remaining columns ({len(remaining_columns)}):")
print(remaining_columns)


In [ ]:
# Fill missing numerical values with median
numerical_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
df_cleaned[numerical_cols] = df_cleaned[numerical_cols].fillna(df_cleaned[numerical_cols].median())

# Fill missing categorical values with mode
categorical_cols = df_cleaned.select_dtypes(include=['object']).columns
df_cleaned[categorical_cols] = df_cleaned[categorical_cols].fillna(df_cleaned[categorical_cols].mode().iloc[0])

# Check for missing values in numerical columns
print("Missing values in numerical columns:")
print(df_cleaned[numerical_cols].isnull().sum())

# Check for missing values in categorical columns
print("Missing values in categorical columns:")
print(df_cleaned[categorical_cols].isnull().sum())



In [ ]:
#Save Cleaned File
df_cleaned.to_csv('cleaned_survey_results.csv', index=False)


CLEANING CODE OF EXPERIENCE

In [ ]:
# # Cleaning Years of Experience YearsCode(overall coding experience) && YearsCodePro (coding experience as a professional)

# # Step 1: Replace 'NA' strings with NaN
# df.replace('NA', pd.NA, inplace=True)

# # Step 2: Convert 'YearsCode' and 'YearsCodePro' columns to numeric, 
# # forcing errors to NaN for any other non-numeric values
# df['YearsCode'] = pd.to_numeric(df['YearsCode'], errors='coerce')
# df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

# # Step 3: Impute missing values with the mean of each column
# df['YearsCode'] = df['YearsCode'].fillna(df['YearsCode'].mean())
# df['YearsCodePro'] = df['YearsCodePro'].fillna(df['YearsCodePro'].mean())

# # Round the results to whole numbers (integers)
# df['YearsCode'] = df['YearsCode'].round().astype(int)
# df['YearsCodePro'] = df['YearsCodePro'].round().astype(int)

# # Print the cleaned DataFrame with the two columns
# # print(df[['YearsCode', 'YearsCodePro']])

CLEANING EDUCATION LEVEL

In [ ]:
# # Clean the EdLevel column:
# # - Remove text in parentheses (e.g., "(e.g. American high school, etc.)")
# # - Strip any extra spaces
# df['EdLevel'] = df['EdLevel'].apply(lambda x: re.sub(r'\(.*\)', '', str(x)).strip())

# # Mapping dictionary for converting text to numeric values
# edlevel_mapping = {
#     'Primary/elementary school': 1,
#     'Secondary school': 2,
#     "Bachelor's degree": 3,
#     'Associate degree': 4,
#     "Master's degree": 5,
#     'Professional degree': 6,
#     'Some college/university study without earning a degree': 0,
#     'Something else': 0
# }

# # Replace text with numeric values according to the mapping
# df['EdLevel'] = df['EdLevel'].replace(edlevel_mapping)

# # Convert the column to numeric values (in case there are still mixed types)
# df['EdLevel'] = pd.to_numeric(df['EdLevel'], errors='coerce')

# # Handle missing values (NaN) and replace with -1
# df['EdLevel'] = df['EdLevel'].fillna(-1)

# # Print only the cleaned 'EdLevel' column
# # print(df['EdLevel'])

CLEANING ORGSIZE

In [ ]:
# # Display unique values in OrgSize and their counts
# print("Unique values in OrgSize before parsing:")
# print(df_cluster['OrgSize'].value_counts(dropna=False))

In [ ]:
# # Function to parse OrgSize values
# def parse_org_size(value):
#     if pd.isnull(value):  # Handle missing values
#         return np.nan
#     value = str(value).lower()
    
#     # Handle ranges like "100 to 499 employees"
#     if "to" in value:
#         try:
#             return int(value.split("to")[0].replace(",", "").strip())
#         except ValueError:
#             return np.nan
    
#     # Handle "10,000 or more employees"
#     elif "or more" in value:
#         try:
#             return int(value.split("or")[0].replace(",", "").strip())
#         except ValueError:
#             return np.nan

#     # Handle freelancer entries
#     elif "just me" in value or "freelancer" in value:
#         return 1  # Single individual

#     # Handle unknown or NA
#     elif "i don't know" in value or value in ["na"]:
#         return np.nan

#     # Default fallback for unexpected values
#     else:
#         return np.nan

# # Display original unique values in OrgSize
# print("Unique values in OrgSize before parsing:")
# print(df_cluster['OrgSize'].value_counts(dropna=False))

# # Apply parsing to OrgSize column
# df_cluster['OrgSize'] = df_cluster['OrgSize'].map(parse_org_size)

# # Fill missing values with the median
# df_cluster['OrgSize'] = df_cluster['OrgSize'].fillna(df_cluster['OrgSize'].median())

# # Display transformed OrgSize values after parsing
# print("\nUnique values in OrgSize after parsing:")
# print(df_cluster['OrgSize'].value_counts())

# # Optional: Display final values after filling missing values
# print("\nUnique values in OrgSize after filling missing values:")
# print(df_cluster['OrgSize'].value_counts())


CLEANING LANGUAGE HAVE WORKED WITH

In [ ]:
# # Handle NaN values by filling them with an empty string or dropping rows with NaN values
# df['LanguageHaveWorkedWith'] = df['LanguageHaveWorkedWith'].fillna('')

# # Split the 'LanguageHaveWorkedWith' column by semicolon
# df['Technologies'] = df['LanguageHaveWorkedWith'].str.split(';')

# # Standardize technology names (strip extra spaces, title case)
# df['Technologies'] = df['Technologies'].apply(lambda x: [tech.strip().title() for tech in x if tech])

# # Flatten the list of technologies and create a set of unique technologies
# all_technologies = set([tech for sublist in df['Technologies'] for tech in sublist])

# # One-hot encode by creating a column for each technology
# for tech in all_technologies:
#     df[tech] = df['Technologies'].apply(lambda x: 1 if tech in x else 0)

# # Drop the original 'LanguageHaveWorkedWith' and 'Technologies' columns
# df.drop(columns=['LanguageHaveWorkedWith', 'Technologies'], inplace=True)

# # Print the one-hot encoded results
# # print(df)

CLEANING DATABASE HAVE WORKED WITH

In [ ]:
# # Handle NaN values by filling them with an empty string or dropping rows with NaN values
# df['DatabaseHaveWorkedWith'] = df['DatabaseHaveWorkedWith'].fillna('')

# # Function to clean and one-hot encode a column
# def clean_and_encode(column_name):
#     # Split the column by semicolon
#     df[column_name + '_Technologies'] = df[column_name].str.split(';')
    
#     # Standardize technology names (strip extra spaces, title case)
#     df[column_name + '_Technologies'] = df[column_name + '_Technologies'].apply(lambda x: [tech.strip().title() for tech in x if tech])
    
#     # Flatten the list of technologies and create a set of unique technologies
#     all_technologies = set([tech for sublist in df[column_name + '_Technologies'] for tech in sublist])
    
#     # One-hot encode by creating a column for each technology
#     for tech in all_technologies:
#         df[tech] = df[column_name + '_Technologies'].apply(lambda x: 1 if tech in x else 0)
    
#     # Drop the original technology columns
#     df.drop(columns=[column_name, column_name + '_Technologies'], inplace=True)

# # Clean and one-hot encode the 'DatabaseHaveWorkedWith' column
# clean_and_encode('DatabaseHaveWorkedWith')

CLEANING DEVTYPE

In [ ]:
# # Replace "NA" with NaN (missing values) in DevType column
# df['DevType'] = df['DevType'].replace('NA', np.nan)

# # Define a mapping to group similar roles
# dev_type_mapping = {
#     'Academic researcher': 'Researcher',
#     'Blockchain': 'Developer',
#     'Cloud infrastructure engineer': 'Engineer',
#     'Data engineer': 'Data Professional',
#     'Data or business analyst': 'Data Professional',
#     'Data scientist or machine learning specialist': 'Data Professional',
#     'Database administrator': 'Data Professional',
#     'Designer': 'Designer',
#     'Developer Advocate': 'Developer',
#     'Developer Experience': 'Developer',
#     'Developer, AI': 'Developer',
#     'Developer back-end': 'Developer',
#     'Developer, desktop or enterprise applications': 'Developer',
#     'Developer, embedded applications': 'Developer',
#     'Developer, front-end': 'Developer',
#     'Developer, full-stack': 'Developer',
#     'Developer, game or graphics': 'Developer',
#     'Developer, mobile': 'Developer',
#     'Developer, QA or test': 'Developer',
#     'DevOps specialist': 'Engineer',
#     'Educator': 'Educator',
#     'Engineer site reliability': 'Engineer',
#     'Engineering manager': 'Manager',
#     'Hardware manager': 'Manager',
#     'Marketing or sales professional': 'Business',
#     'Others': 'Other',
#     'Product manager': 'Manager',
#     'Project manager': 'Manager',
#     'Research and Development role': 'Researcher',
#     'Scientist': 'Researcher',
#     'Security professional': 'Security',
#     'Senior Executive (C-Suite, VP)': 'Executive',
#     'Student': 'Student',
#     'System administrator': 'Engineer'
# }

# # Apply the mapping
# df['DevType'] = df['DevType'].map(dev_type_mapping)

# # Convert categorical column to numeric using a mapping for the grouped roles
# dev_type_numeric_mapping = {
#     'Researcher': 1,
#     'Developer': 2,
#     'Engineer': 3,
#     'Data Professional': 4,
#     'Designer': 5,
#     'Manager': 6,
#     'Business': 7,
#     'Other': 8,
#     'Educator': 9,
#     'Security': 10,
#     'Executive': 11,
#     'Student': 12
# }

# # Apply the numeric mapping to the 'DevType' column
# df['DevTypeNumeric'] = df['DevType'].map(dev_type_numeric_mapping)

# # Fill NaN values (optional, depending on your analysis)
# df['DevTypeNumeric'] = df['DevTypeNumeric'].fillna(-1)  # Use -1 or another placeholder for missing values


##### Exploratory Data Analysis (EDA)

In [197]:
# place code here

##### Data Analysis Techniques

In [198]:
# K-means Clustering

In [199]:
# Linear Regression

In [200]:
# Apriori Algorithm